In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/books_data.csv')
ratings = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/Books_rating.csv')

In [3]:
data.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [4]:
ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil

# Load the dataset
df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/Books_rating.csv')

# Display the first few rows
df.head()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [2]:
# columns available in the DataFrame
print("Available columns in the DataFrame:", df.columns)

Available columns in the DataFrame: Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')


In [3]:
# Define a function to safely parse the helpfulness ratio
def parse_helpfulness_ratio(helpfulness):
    try:
        votes = list(map(int, helpfulness.split('/')))
        if len(votes) == 2 and votes[1] != 0:
            return votes[0] / votes[1]
    except:
        return np.nan
    return np.nan

# Calculate helpfulness ratio using the safe function
df['helpfulness_ratio'] = df['review/helpfulness'].apply(parse_helpfulness_ratio)

# Ensure all required columns are available before dropping NaNs
required_columns = ['review/score', 'Price', 'helpfulness_ratio', 'review/text']

# Check for missing values specifically in these columns
missing_data = df[required_columns].isnull().sum()
print("Missing values in each column before dropna:")
print(missing_data)

# Drop rows with missing values in these columns
df.dropna(subset=required_columns, inplace=True)

# Verify that the required columns have no missing values after cleaning
missing_data_after = df[required_columns].isnull().sum()
print("Missing values in each column after dropna:")
print(missing_data_after)

# Display the first few rows after cleaning
df.head()


Missing values in each column before dropna:
review/score               0
Price                2518829
helpfulness_ratio     885732
review/text                8
dtype: int64
Missing values in each column after dropna:
review/score         0
Price                0
helpfulness_ratio    0
review/text          0
dtype: int64


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,helpfulness_ratio
10,0829814000,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP,Rev. Pamela Tinnin,8/10,5.0,991440000,Outstanding Resource for Small Church Pastors,"I just finished the book, &quot;Wonderful Wors...",0.800000
11,0829814000,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N,Dr. Terry W. Dorsett,1/1,5.0,1291766400,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...,1.000000
12,0829814000,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4,"Cynthia L. Lajoy ""Cindy La Joy""",1/1,5.0,1248307200,Not Just for Pastors!,I just finished reading this amazing book and ...,1.000000
13,0829814000,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y,Maxwell Grant,1/1,5.0,1222560000,Small church pastor? This is the book on worship,I hadn't been a small church pastor very long ...,1.000000
14,0595344550,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB,Book Reader,7/11,1.0,1117065600,not good,I bought this book because I read some glowing...,0.636364


In [4]:
df.shape

(356273, 11)

In [5]:
# Fill NaN values in text columns with empty strings
df['review/text'].fillna('', inplace=True)
df['review/summary'].fillna('', inplace=True)

# Function for frequency encoding
def frequency_encoding(column):
    freq = column.value_counts() / len(column)
    return column.map(freq)

# Apply frequency encoding to the text columns
df['review_text_encoded'] = frequency_encoding(df['review/text'])
df['review_summary_encoded'] = frequency_encoding(df['review/summary'])

# Combine the encoded features with other numeric features
processed_df = pd.concat([df[['Price', 'helpfulness_ratio', 'review_text_encoded', 'review_summary_encoded']]], axis=1)

# Verify the shape of the processed DataFrame
print("Processed DataFrame shape:", processed_df.shape)
processed_df.head()

Processed DataFrame shape: (356273, 4)


,Price,helpfulness_ratio,review_text_encoded,review_summary_encoded
10,19.40,0.800000,0.000003,0.000003
11,19.40,1.000000,0.000003,0.000003
12,19.40,1.000000,0.000003,0.000003
13,19.40,1.000000,0.000003,0.000003
14,10.95,0.636364,0.000003,0.000031


In [6]:
# Define features (X) and target (y)
X = processed_df
y = df['review/score']

# Display the shapes of X and y
print(f'X shape: {X.shape}, y shape: {y.shape}')

X shape: (356273, 4), y shape: (356273,)


In [7]:
# Function to sample data (updated to handle full dataset properly)
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            # Use the entire dataset for training, leave a small portion for testing
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [14]:
# Function to calculate and return metrics
from sklearn.tree import DecisionTreeRegressor


def calculate_metrics(X_train, X_test, y_train, y_test):
    dt = DecisionTreeRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'max_depth': [10, 20, 30, 50, 75, 100],
        'min_samples_split': [2, 5, 10, 20, 30],
        'min_samples_leaf': [5, 10, 20, 50],
        'max_features': ['sqrt', 'log2', None]
    }

    random_search = RandomizedSearchCV(dt, param_distributions, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=2)
    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [15]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Metrics for sample size 1.0:
RMSE: 1.1206231339131583
MAPE: 0.35568912468164476
R2: 0.19160148826824086
MSE: 1.2557962082613483
nRMSE: 0.28015578347828957
Execution Time (Raw): 60.21718907356262
Normalized Time (s/MB): 4.435196735325189
Average CPU Usage: 50.15
Sample Size: 1.0
--------------------------------------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Metrics for sample size 0.5:
RMSE: 1.1699788093656367
MAPE: 0.3961143505267879
R2: 0.18298019808328747
MSE: 1.3688504143646327
nRMSE: 0.2924947023414092
Execution Time (Raw): 25.572787284851074
Normalized Time (s/MB): 3.763277916872502
Average CPU Usage: 53.300000000000004
Sample Size: 0.5
--------------------------------------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Metrics for sample size 0.25:
RMSE: 1.1772466131394808
MAPE: 0.40053785280494314
R2: 0.17279816311482232
MSE: 1.3859095881483785
nRMSE: 0.2

In [16]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 1 minutes and 59.02 seconds
Total Normalized Execution Time for Entire Process: 4.12423456 seconds per MB
Total Average CPU Usage for Entire Process: 42.52%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,1.120623,0.355689,0.191601,1.255796,0.280156,60.217189,4.435197,50.15,1.000
1,1.169979,0.396114,0.182980,1.368850,0.292495,25.572787,3.763278,53.30,0.500
2,1.177247,0.400538,0.172798,1.385910,0.294312,13.353613,3.930221,43.00,0.250
3,1.185101,0.403902,0.161723,1.404465,0.296275,6.013762,3.539928,51.40,0.125
4,0.911109,0.214298,0.087780,0.830120,0.227777,0.582838,190.984453,24.70,100.000
5,1.313604,0.504800,0.054167,1.725555,0.328401,0.623020,20.415125,24.25,1000.000
6,1.240247,0.446017,0.155533,1.538213,0.310062,1.231145,4.034215,47.50,10000.000
7,1.177856,0.399374,0.164523,1.387345,0.294464,11.428918,3.745028,45.90,100000.000
